In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import tarfile
import pandas as pd
import os
import re
from torch.utils.data import Dataset, DataLoader, ConcatDataset, random_split
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from sklearn.metrics import confusion_matrix
from io import StringIO
from sklearn.decomposition import PCA

In [2]:
class DatasetClass(Dataset):
    
    def __init__(self, folder, filename, label_dict):
        
        self.data = []
        self.filename = filename
        tar = tarfile.open(folder + '\\' + filename)
        for file in tar.getmembers():
            f = tar.extractfile(file)
            if f != None:
                content = pd.read_csv(StringIO(f.read().decode()), sep=' ', header=None).values.ravel()
                self.data.append(content)
            
        self.y = torch.tensor(label_dict[self.filename[:-7]], dtype=torch.long)
    
    def __getitem__(self, idx):     
        
        return torch.tensor(self.data[idx], dtype=torch.float), self.y
      
    def __len__(self):
        
        return len(self.data)

In [3]:
def train_test_loader(directory, label_dict, train_fraction=0.8, num_workers=2, batch_size=32):

    all_files = list(filter(lambda x: x.endswith('.tar.gz'), os.listdir(directory)))
    files = [file for file in all_files if file[:-7] in label_dict.keys()]
    
    datasets = list(map(lambda x : DatasetClass(directory, x, label_dict), files))
    dataset = ConcatDataset(datasets)
    N = dataset.cumulative_sizes[-1]
    
    train_size = int(N*train_fraction)
    test_size = N - train_size

    train_data, test_data = torch.utils.data.random_split(dataset, [train_size, test_size])

    trainloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    testloader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    
    return trainloader, testloader

In [4]:
label_dict = {'tallbuilding': 0, 'opencountry':1, 'mountain': 2, 'highway': 3, 'coast': 4}
trainloader, testloader = train_test_loader('Data_Set_1(Colored_Images)', label_dict, train_fraction=0.8, num_workers=0)

In [5]:
class AutoEncoder(nn.Module):
    
    def __init__(self, n_features, h_layer_sizes):
        super(AutoEncoder, self).__init__()
        
        self.fc1 = nn.Linear(n_features, h_layer_sizes[0])
        self.fc2 = nn.Linear(h_layer_sizes[0], h_layer_sizes[1])
        self.fc3 = nn.Linear(h_layer_sizes[1], h_layer_sizes[2])
        self.out = nn.Linear(h_layer_sizes[2], n_features)
        
    def forward(self, x):
        
        x = torch.tanh(self.fc1(x)) # Hidden Layer 1 (Tanh)
        x = self.fc2(x)    # Hidden Layer 2 (Linear)
        x = torch.tanh(self.fc3(x)) # Hidden Layer 3 (Tanh)
        x = self.out(x) # Output Layer (Linear)
        
        return x
    
    def get_z(self, x):
        
        z = torch.tanh(self.fc1(x))
        z = self.fc2(z)
        
        return z

In [24]:
criterion = nn.MSELoss()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [25]:
ae1 = AutoEncoder(828, [500, 300, 500])
ae1 = ae1.to(device)
optimizer1 = optim.SGD(ae1.parameters(), lr=0.0001, momentum=0.9)

In [26]:
old_loss = np.inf

max_epoch = 500

for epoch in range(max_epoch):

    running_loss = 0.0
    
    for data in trainloader:
        
        X, _ = data[0].to(device), data[1].to(device)
        
        optimizer1.zero_grad()
        
        # Reconstructed Representation of X (forward)
        X_hat = ae1(X)
        
        # Calculate Loss (MSE)
        loss = criterion(X_hat, X)
        
        # Backpropagation
        loss.backward()
        
        # Update Parameters
        optimizer1.step()
        
        running_loss += loss.item()
    
    print('Epoch', epoch+1, ': Loss =', running_loss)
    
    
    if abs(running_loss-old_loss)/running_loss < 1e-4:
        print('Converged')
        break
    
    old_loss = running_loss

print('Finished Training')

Epoch 1 : Loss = 84.81388974189758
Epoch 2 : Loss = 83.72193026542664
Epoch 3 : Loss = 82.63125085830688
Epoch 4 : Loss = 81.56800734996796
Epoch 5 : Loss = 80.50918519496918
Epoch 6 : Loss = 79.4323400259018
Epoch 7 : Loss = 78.31987202167511
Epoch 8 : Loss = 77.15656733512878
Epoch 9 : Loss = 75.92866683006287
Epoch 10 : Loss = 74.62778151035309
Epoch 11 : Loss = 73.24534940719604
Epoch 12 : Loss = 71.7760180234909
Epoch 13 : Loss = 70.21788251399994
Epoch 14 : Loss = 68.57295835018158
Epoch 15 : Loss = 66.8405749797821
Epoch 16 : Loss = 65.02913665771484
Epoch 17 : Loss = 63.14270734786987
Epoch 18 : Loss = 61.19146478176117
Epoch 19 : Loss = 59.187352657318115
Epoch 20 : Loss = 57.136245012283325
Epoch 21 : Loss = 55.05608332157135
Epoch 22 : Loss = 52.95425021648407
Epoch 23 : Loss = 50.848626494407654
Epoch 24 : Loss = 48.74791431427002
Epoch 25 : Loss = 46.66660034656525
Epoch 26 : Loss = 44.615538418293
Epoch 27 : Loss = 42.6039200425148
Epoch 28 : Loss = 40.6421115398407
Epoch

In [28]:
class FinalNet(nn.Module):
    
    def __init__(self, input_size, hidden_sizes, num_classes):
        super(FinalNet, self).__init__()
        
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
        #self.fc4 = nn.Linear(hidden_sizes[2], hidden_sizes[3])
        self.out = nn.Linear(hidden_sizes[2], num_classes)
    
    def forward(self, x):
        
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        #x = torch.tanh(self.fc4(x))
        x = self.out(x)
        
        return x
    
    def predict(self, X):
        
        with torch.no_grad():
            y_score = self.forward(X)
            y_pred = torch.argmax(y_score, axis=1)
            
        return y_pred
            
    
classifier = FinalNet(300, [150, 75, 50], 5)

In [72]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(classifier.parameters(), lr=0.00001, momentum=0.9)
classifier = classifier.to(device)

In [73]:
old_loss = np.inf

max_epoch = 500

for epoch in range(max_epoch):

    running_loss = 0.0
    
    for data in trainloader:
        
        X, y = data[0].to(device), data[1].to(device)
        
        # extracting encoder features from AE1 to use as input to the MLFFNN
        with torch.no_grad():
            Z = ae1.get_z(X)
        
        optimizer.zero_grad()
        
        # Forward
        y_hat = classifier(Z)
        
        # Calculate Loss (Cross Entropy)
        loss = criterion(y_hat, y)
        
        # Backpropagation
        loss.backward()
        
        # Update Parameters
        optimizer.step()
        
        running_loss += loss.item()
    
    print('Epoch', epoch+1, ': Loss =', running_loss)
    
    if abs(running_loss-old_loss)/running_loss < 1e-6:
        print('Converged')
        break
    
    old_loss = running_loss

print('Finished Training')

Epoch 1 : Loss = 3.2300708070397377
Epoch 2 : Loss = 3.1674853786826134
Epoch 3 : Loss = 3.1596838608384132
Epoch 4 : Loss = 3.160363093018532
Epoch 5 : Loss = 3.156294383108616
Epoch 6 : Loss = 3.1562704667448997
Epoch 7 : Loss = 3.1523110792040825
Epoch 8 : Loss = 3.1593799591064453
Epoch 9 : Loss = 3.1622075363993645
Epoch 10 : Loss = 3.1569722816348076
Epoch 11 : Loss = 3.1562914103269577
Epoch 12 : Loss = 3.154237687587738
Epoch 13 : Loss = 3.1552461609244347
Epoch 14 : Loss = 3.1530437394976616
Epoch 15 : Loss = 3.1548278108239174
Epoch 16 : Loss = 3.155103400349617
Epoch 17 : Loss = 3.155000627040863
Epoch 18 : Loss = 3.156525269150734
Epoch 19 : Loss = 3.1561090648174286
Epoch 20 : Loss = 3.151229351758957
Epoch 21 : Loss = 3.1527416929602623
Epoch 22 : Loss = 3.1551481038331985
Epoch 23 : Loss = 3.152328483760357
Epoch 24 : Loss = 3.1599630266427994
Epoch 25 : Loss = 3.1577599570155144
Epoch 26 : Loss = 3.1541131362318993
Epoch 27 : Loss = 3.1522035598754883
Epoch 28 : Loss = 

KeyboardInterrupt: 

In [69]:
with torch.no_grad():
    
    test_loss = 0.0
    y_test = []
    y_test_pred = []

    for data in trainloader:

        X, y = data[0].to(device), data[1].to(device)
        Z = ae1.get_z(X)
        y_hat = classifier(Z)      
        test_loss += criterion(y_hat, y)
        
        y_test.extend(list(y.cpu().detach().numpy()))
        y_test_pred.extend(list(torch.argmax(y_hat, axis=1).cpu().detach().numpy()))

print('Train Loss =', test_loss.item())
pd.DataFrame(confusion_matrix(y_test, y_test_pred))

Train Loss = 3.288172483444214


,0,1,2,3,4
0,289,0,1,0,0
1,0,329,2,0,0
2,1,6,281,0,0
3,0,0,0,209,0
4,0,3,2,0,285


In [70]:
with torch.no_grad():
    
    test_loss = 0.0
    y_test = []
    y_test_pred = []

    for data in testloader:

        X, y = data[0].to(device), data[1].to(device)
        Z = ae1.get_z(X)
        y_hat = classifier(Z)      
        test_loss += criterion(y_hat, y)
        
        y_test.extend(list(y.cpu().detach().numpy()))
        y_test_pred.extend(list(torch.argmax(y_hat, axis=1).cpu().detach().numpy()))

print('Test Loss =', test_loss.item())
pd.DataFrame(confusion_matrix(y_test, y_test_pred))

Test Loss = 30.625680923461914


,0,1,2,3,4
0,26,8,20,8,4
1,3,44,12,5,15
2,17,17,36,6,10
3,1,6,6,26,12
4,6,16,12,8,28


In [71]:
from sklearn.metrics import accuracy_score, f1_score, precision_score
acc = accuracy_score(y_test, y_test_pred)
prec = precision_score(y_test, y_test_pred, average='macro')
f1 = f1_score(y_test, y_test_pred, average='macro')

print('Test Accuracy =', acc, 'Test Precision =', prec, 'Test F1 =', f1)

Test Accuracy = 0.45454545454545453 Test Precision = 0.45781006232004956 Test F1 = 0.45522083954880266


In [67]:
torch.save(classifier.state_dict(), 'a2_q1_wts.pt')

# PCA

In [14]:
# To fit the PCA model we load all the training points in a single batch
train, test = train_test_loader('Data_Set_1(Colored_Images)', label_dict, train_fraction=0.8, batch_size=2000, num_workers=0)

for i in train:
    print(i[0])
    temp = i[0]
    print(i[0].shape)
    break

tensor([[0.1624, 0.4426, 0.6126,  ..., 3.2938, 3.3324, 3.2227],
        [0.5676, 0.5882, 0.5283,  ..., 0.4174, 0.4174, 0.4144],
        [0.5766, 0.1428, 0.9406,  ..., 2.4045, 2.5507, 2.7237],
        ...,
        [0.5540, 0.3268, 0.7248,  ..., 3.1994, 3.3497, 3.1775],
        [0.6029, 0.3759, 0.8112,  ..., 2.7578, 2.8919, 2.9956],
        [0.5877, 0.4436, 0.7396,  ..., 3.6991, 3.4472, 3.2400]])
torch.Size([1408, 828])


In [15]:
def return_pca(temp):
    pca = PCA(n_components=0.99)
    pca.fit(temp)
    return pca

PCA_model = return_pca(temp)

In [16]:
reduced_dimension = PCA_model.transform(temp).shape[1]
pca_clf = FinalNet(reduced_dimension, [150, 75, 50], 5)

In [17]:
reduced_dimension

295

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pca_clf.parameters(), lr=0.001, momentum=0.9)
pca_clf = pca_clf.to(device)

In [19]:
old_loss = np.inf

max_epoch = 500

for epoch in range(max_epoch):

    running_loss = 0.0
    
    for data in trainloader:
        
        X, y = data[0], data[1].to(device)
        
        # applying PCA on the data to use as input to the MLFFNN
        Z = torch.Tensor(PCA_model.transform(X)).to(device)
        
        optimizer.zero_grad()
        
        # Forward
        y_hat = pca_clf(Z)
        
        # Calculate Loss (Cross Entropy)
        loss = criterion(y_hat, y)
        
        # Backpropagation
        loss.backward()
        
        # Update Parameters
        optimizer.step()
        
        running_loss += loss.item()
    
    print('Epoch', epoch+1, ': Loss =', running_loss)
    
    if abs(running_loss-old_loss)/running_loss < 1e-4:
        print('Converged')
        break
    
    old_loss = running_loss

print('Finished Training')

Epoch 1 : Loss = 70.05836939811707
Epoch 2 : Loss = 68.88779950141907
Epoch 3 : Loss = 67.606889128685
Epoch 4 : Loss = 66.15462255477905
Epoch 5 : Loss = 64.5316731929779
Epoch 6 : Loss = 62.834609031677246
Epoch 7 : Loss = 61.232816100120544
Epoch 8 : Loss = 59.77930772304535
Epoch 9 : Loss = 58.445438265800476
Epoch 10 : Loss = 57.12319362163544
Epoch 11 : Loss = 55.81054937839508
Epoch 12 : Loss = 54.393919229507446
Epoch 13 : Loss = 52.95768928527832
Epoch 14 : Loss = 51.49075174331665
Epoch 15 : Loss = 50.04957616329193
Epoch 16 : Loss = 48.61212873458862
Epoch 17 : Loss = 47.21941953897476
Epoch 18 : Loss = 45.86977535486221
Epoch 19 : Loss = 44.59262466430664
Epoch 20 : Loss = 43.32969403266907
Epoch 21 : Loss = 42.130806386470795
Epoch 22 : Loss = 41.031650483608246
Epoch 23 : Loss = 39.95331799983978
Epoch 24 : Loss = 38.95661270618439
Epoch 25 : Loss = 37.99318724870682
Epoch 26 : Loss = 37.099294781684875
Epoch 27 : Loss = 36.247111320495605
Epoch 28 : Loss = 35.43063855171

Epoch 222 : Loss = 0.3308044373989105
Epoch 223 : Loss = 0.32822395861148834
Epoch 224 : Loss = 0.32438911497592926
Epoch 225 : Loss = 0.3215700536966324
Epoch 226 : Loss = 0.31803208589553833
Epoch 227 : Loss = 0.31501244008541107
Epoch 228 : Loss = 0.3121921122074127
Epoch 229 : Loss = 0.30918169021606445
Epoch 230 : Loss = 0.3059910237789154
Epoch 231 : Loss = 0.30343766510486603
Epoch 232 : Loss = 0.300594687461853
Epoch 233 : Loss = 0.297659695148468
Epoch 234 : Loss = 0.2951820641756058
Epoch 235 : Loss = 0.2928411066532135
Epoch 236 : Loss = 0.28990454971790314
Epoch 237 : Loss = 0.2870728373527527
Epoch 238 : Loss = 0.28491948544979095
Epoch 239 : Loss = 0.2825154811143875
Epoch 240 : Loss = 0.27966585755348206
Epoch 241 : Loss = 0.27732308208942413
Epoch 242 : Loss = 0.27520811557769775
Epoch 243 : Loss = 0.27278752624988556
Epoch 244 : Loss = 0.27018335461616516
Epoch 245 : Loss = 0.26811665296554565
Epoch 246 : Loss = 0.26558053493499756
Epoch 247 : Loss = 0.2635371685028076

Epoch 434 : Loss = 0.09583932161331177
Epoch 435 : Loss = 0.09549792110919952
Epoch 436 : Loss = 0.09516498446464539
Epoch 437 : Loss = 0.09478707611560822
Epoch 438 : Loss = 0.09450791776180267
Epoch 439 : Loss = 0.09412683546543121
Epoch 440 : Loss = 0.09379677474498749
Epoch 441 : Loss = 0.0934617817401886
Epoch 442 : Loss = 0.09310056269168854
Epoch 443 : Loss = 0.09276877343654633
Epoch 444 : Loss = 0.09244254231452942
Epoch 445 : Loss = 0.09211806952953339
Epoch 446 : Loss = 0.09178347885608673
Epoch 447 : Loss = 0.09147010743618011
Epoch 448 : Loss = 0.09114730358123779
Epoch 449 : Loss = 0.09081460535526276
Epoch 450 : Loss = 0.09050148725509644
Epoch 451 : Loss = 0.09020787477493286
Epoch 452 : Loss = 0.08987978100776672
Epoch 453 : Loss = 0.0895673930644989
Epoch 454 : Loss = 0.08925142884254456
Epoch 455 : Loss = 0.08897152543067932
Epoch 456 : Loss = 0.0886412262916565
Epoch 457 : Loss = 0.08833770453929901
Epoch 458 : Loss = 0.08802661299705505
Epoch 459 : Loss = 0.0877333

In [74]:
with torch.no_grad():
    
    test_loss = 0.0
    y_test = []
    y_test_pred = []

    for data in trainloader:

        X, y = data[0], data[1].to(device)
        Z = torch.Tensor(PCA_model.transform(X)).to(device)
        y_hat = pca_clf(Z)      
        test_loss += criterion(y_hat, y)
        
        y_test.extend(list(y.cpu().detach().numpy()))
        y_test_pred.extend(list(torch.argmax(y_hat, axis=1).cpu().detach().numpy()))

print('Train Loss =', test_loss.item())
pd.DataFrame(confusion_matrix(y_test, y_test_pred))

Train Loss = 0.07644738256931305


,0,1,2,3,4
0,290,0,0,0,0
1,0,331,0,0,0
2,0,0,288,0,0
3,0,0,0,209,0
4,0,0,0,0,290


In [20]:
with torch.no_grad():
    
    test_loss = 0.0
    y_test = []
    y_test_pred = []

    for data in testloader:

        X, y = data[0], data[1].to(device)
        Z = torch.Tensor(PCA_model.transform(X)).to(device)
        y_hat = pca_clf(Z)      
        test_loss += criterion(y_hat, y)
        
        y_test.extend(list(y.cpu().detach().numpy()))
        y_test_pred.extend(list(torch.argmax(y_hat, axis=1).cpu().detach().numpy()))

print('Test Loss =', test_loss.item())
pd.DataFrame(confusion_matrix(y_test, y_test_pred))

Test Loss = 30.87776756286621


,0,1,2,3,4
0,39,6,14,3,4
1,4,46,4,3,22
2,12,11,45,7,11
3,5,4,5,32,5
4,3,16,10,6,35


In [22]:
from sklearn.metrics import accuracy_score, f1_score, precision_score
acc = accuracy_score(y_test, y_test_pred)
prec = precision_score(y_test, y_test_pred, average='macro')
f1 = f1_score(y_test, y_test_pred, average='macro')

print('Test Accuracy =', acc, 'Test Precision =', prec, 'Test F1 =', f1)

Test Accuracy = 0.5596590909090909 Test Precision = 0.5664367996756374 Test F1 = 0.5649948683492221
